# Загрузка Pandas и очистка данных

In [11]:
import pandas as pd
import numpy as np
import re
import datetime

In [4]:
data = pd.read_csv('main_task.csv')
display(data)
print(data.info())

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB
None


In [5]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки
#средним
data['Number of Reviews'].fillna(int(data['Number of Reviews'].mean()), inplace = True)


In [6]:
# City - оставим 10 часто встречающихся городов, остальные города заменем на 'other'
all_cities = data.City.value_counts().index
top_cities = list(data.City.value_counts().sort_values(
    ascending=False).head(11).index)
cities_to_throw_away = list(set(all_cities) - set(top_cities))
data.loc[data['City'].isin(cities_to_throw_away), 'City'] = 'other'

# создали переменные dummies
dum = pd.get_dummies(data.City, drop_first=True)

# присоединили к данным

data = pd.concat([data, dum], axis=1)

In [7]:
#Заполняем пустые значения колонки Price Range самой распространенной ценовой категорией
data['Price Range'] = data['Price Range'].fillna('$$ - $$$')
#Заменяем значения на числа
def price_range_int(range):
    if range == '$':
        return 1
    elif range == '$$ - $$$':
        return 2
    elif range == '$$$$':
        return 3
    else:
        return 0
data['Price Range'] = data['Price Range'].apply(price_range_int)
data['Price Range'].value_counts()

2    32298
1     6279
3     1423
Name: Price Range, dtype: int64

In [8]:
# рестораны с одинаковыми id-скорее всего это сетевые рестораны, введем новый признак net

rests = data['Restaurant_id'].value_counts()
print(rests)
data['net_size'] = data['Restaurant_id'].apply(lambda x: rests[x])
data['net'] = data['net_size'].apply(lambda x: 0 if x<3 else 1)

display(data['net'].value_counts())
display(data['net_size'].value_counts())

id_227      18
id_436      18
id_871      18
id_633      18
id_430      17
            ..
id_16068     1
id_13134     1
id_16019     1
id_13286     1
id_15896     1
Name: Restaurant_id, Length: 11909, dtype: int64


1    30661
0     9339
Name: net, dtype: int64

1     4705
2     4634
3     3678
4     3248
9     2988
7     2688
5     2570
10    2550
8     2488
6     2478
11    2431
12    1812
13    1677
14    1036
15     555
16     288
17     102
18      72
Name: net_size, dtype: int64

In [12]:
# найдем разницу в датах(интервал между отзывами)
data["Reviews"] = data["Reviews"].str.replace("nan", "' '")
data["Reviews"].fillna("[[], []]", inplace=True)

rx = r'(\d\d/\d\d/\d{4})'

data['reviews_Date'] = data["Reviews"].str.findall(rx).apply(','.join)

new_df = data['reviews_Date'].str.split(',',expand=True)
new_df.columns = ['reviews_Date1','reviews_Date2']
new_df['reviews_Date1'] = pd.to_datetime(new_df['reviews_Date1'], utc=True)
new_df['reviews_Date2'] = pd.to_datetime(new_df['reviews_Date2'], errors='ignore', utc=True)
new_df['reviews_Date2'] = new_df['reviews_Date2'].dt.date
new_df['reviews_Date1'] = new_df['reviews_Date1'].dt.date
new_df['Interval_Date'] = (new_df['reviews_Date1'] - new_df['reviews_Date2'])
new_df['Interval_Date'].fillna(value=pd.Timedelta(seconds=0), inplace=True)
new_df['Interval_Date']=(new_df['Interval_Date'] / np.timedelta64(1, 'D')).astype(int) 
new_df.sample(4)

,reviews_Date1,reviews_Date2,Interval_Date
24659,2018-01-03,2018-01-02,1
17326,2017-05-07,2017-04-20,17
31883,2017-11-06,2017-09-19,48
25210,2016-04-25,2016-04-23,2


In [13]:
data = pd.concat((data, new_df), axis = 1)
data.drop(['reviews_Date'],axis=1, inplace=True)
data.sample(4)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,...,Paris,Prague,Rome,Vienna,other,net_size,net,reviews_Date1,reviews_Date2,Interval_Date
9050,id_1971,other,"['Belgian', 'European']",1972.0,4.0,2,5.0,"[['Good Belgian restaurant', 'Good first appro...",/Restaurant_Review-g950780-d10167331-Reviews-B...,d10167331,...,0,0,0,0,1,7,1,2017-11-30,2016-04-21,588
37595,id_1016,other,NaN,1025.0,4.5,2,23.0,"[['Mmmmmmhh: great sauces!', 'Please , someone...",/Restaurant_Review-g187309-d7886703-Reviews-Ve...,d7886703,...,0,0,0,0,1,9,1,2016-12-31,2015-07-09,541
19118,id_1595,Barcelona,"['Mediterranean', 'Spanish', 'Healthy', 'Veget...",1596.0,5.0,2,24.0,"[['Hidden Gem, A Must Visit', 'Great find!'], ...",/Restaurant_Review-g187497-d11929393-Reviews-L...,d11929393,...,0,0,0,0,0,9,1,2017-11-28,2017-10-12,47
16565,id_2332,other,NaN,2333.0,3.0,2,26.0,"[[], []]",/Restaurant_Review-g661468-d8669878-Reviews-Co...,d8669878,...,0,0,0,0,1,5,1,NaT,NaT,0


In [14]:
data['Cuisine Style'].fillna('[unknown]', inplace=True)
data['Cuisine Style'] = data['Cuisine Style'].str.replace('[','')
data['Cuisine Style'] = data['Cuisine Style'].str.replace(']','')
data['Cuisine Style'] = data['Cuisine Style'].str.replace("'",'')
data['Cuisine Style'] = data['Cuisine Style'].str.replace(",",'|')
len(data['Cuisine Style'].values[0].split('|'))
data['Cuisine Style'][0]

'European| French| International'

In [15]:
def num_1 (x):
    y = 0
    return y

data['Cuisine_num'] = data.apply(num_1, axis =1)
data['Cuisine_num'] = data.Cuisine_num.apply(lambda x: len(data['Cuisine Style'][x].split('|')))
data['Cuisine_num'][0]

3

In [16]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [17]:
data.sample(10)

,Ranking,Rating,Price Range,Number of Reviews,Barcelona,Berlin,Lisbon,London,Madrid,Milan,Paris,Prague,Rome,Vienna,other,net_size,net,Interval_Date,Cuisine_num
5290,6109.0,3.5,2,149.0,0,0,0,0,0,0,1,0,0,0,0,5,1,0,3
35210,665.0,4.5,2,159.0,1,0,0,0,0,0,0,0,0,0,0,8,1,21,3
16155,4158.0,5.0,2,124.0,0,0,0,0,1,0,0,0,0,0,0,6,1,0,3
33358,730.0,5.0,1,50.0,0,0,0,0,0,1,0,0,0,0,0,8,1,0,3
16328,1191.0,4.5,2,19.0,0,0,0,0,0,0,0,0,0,0,1,13,1,0,3
9362,15902.0,2.0,2,23.0,0,0,0,1,0,0,0,0,0,0,0,1,0,22,3
8086,3901.0,3.0,2,12.0,0,0,0,0,0,0,0,1,0,0,0,3,1,-273,3
4672,7299.0,3.0,3,5.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,3
38662,1448.0,3.5,2,37.0,0,0,0,0,0,0,0,0,0,0,1,10,1,0,3
17446,3479.0,4.5,2,64.0,0,0,0,0,0,0,1,0,0,0,0,6,1,224,3


In [8]:
#На первом этапе для создания такого датафрейма давайте просто удалим столбцы, содержащие 
#данные типа object, и заполним пропущенные значения (None или NaN) каким-то одним значением 
#(нулём или средним арифметическим) для  всего столбца.
#df.drop(['City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA',
#           'ID_TA'], axis=1, inplace=True)
#df['Number of Reviews'].fillna(int(df['Number of Reviews'].mean()), inplace = True)
#print(df.isna().sum())
#print(df.info())

Restaurant_id        0
Ranking              0
Rating               0
Number of Reviews    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   Ranking            40000 non-null  float64
 2   Rating             40000 non-null  float64
 3   Number of Reviews  40000 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1.2+ MB
None


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [18]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = data.drop(['Rating'], axis = 1)
y = data['Rating']

In [19]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [20]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [21]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [22]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [23]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.25990255714285715
